# Net

> TODO fill in description

In [1]:
#| default_exp net

In [14]:
#| hide
from nbdev.showdoc import *; import nbdev; nbdev.nbdev_export()

In [3]:
#|export
import asyncio
from collections import deque

import fbdev
from fbdev.utils import StateHandler, StateView, StateCollection, EventHandler, EventCollection, AttrContainer, await_multiple_events
from fbdev.packet import Packet
from fbdev.component import BaseComponent, PortSpec, BasePort, InputPort, OutputPort, PortCollection, PortType
from fbdev.graph import EdgeSpec, NodeSpec, Graph, ReadonlyGraph

In [4]:
#|hide
show_doc(fbdev.net.Edge)

---

### Edge

>      Edge (input_node, output_node, maxsize:int)

*Initialize self.  See help(type(self)) for accurate signature.*

In [5]:
#|export
class Edge:
    def __init__(self,
                 input_node,
                 output_node,
                 maxsize:int):
        self._id = None  # TODO: implement id
        self._tail_node = input_node
        self._head_node = output_node
        self._is_enabled = StateHandler("Edge.is_enabled", True)
         
        self._is_empty = StateHandler("Edge.is_empty", True)
        self._is_full = StateHandler("Edge.is_full", False)
        self._is_empty_view = StateView(self._is_empty)
        self._is_full_view = StateView(self._is_full)
        self.has_changed = EventHandler("Edge.has_changed")
        
        self._maxsize = maxsize
        self._packets = deque(maxlen=maxsize)

    @property
    def id(self): return self._id        
    @property
    def tail_node(self): return self._tail_node
    @property
    def head_node(self): return self._head_node
    @property
    def is_enabled(self): return self._is_enabled
    @property
    def is_empty(self): return self._is_empty_view
    @property
    def is_full(self): return self._is_full_view
    @property
    def maxsize(self): return self._maxsize
        
    def num_packets(self): return len(self._packets)
        
    def _update_status_events(self):        
        if len(self._packets) > 0: self._is_empty.set(False)
        else: self._is_empty.set(True)
        if len(self._packets) == self.maxsize and self.maxsize is not None: self._is_full.set(True)
        else: self._is_full.set(False)
    
    def _load(self, packet:Packet):
        if self._is_full.get(): raise RuntimeError("Port is full.")
        self._packets.append(packet)
        self._update_status_events()
        self.has_changed._trigger()
        
    def _unload(self):
        if self._is_empty.get(): raise RuntimeError("Port is empty.")
        packet = self._packets.popleft()
        self._update_status_events()
        self.has_changed._trigger()
        return packet
    
    def _load_left(self, packet: Packet):
        if self._is_full.get(): raise RuntimeError("Port is full.")
        self._packets.appendleft(packet)
        self._update_status_events()
        self.has_changed._trigger()
    
    def _unload_right(self):
        if self._is_empty.get(): raise RuntimeError("Port is empty.")
        packet = self._packets.pop()
        self._update_status_events()
        self.has_changed._trigger()
        return packet
        
    def _unload_all(self):
        if self._is_empty.get(): raise RuntimeError("Port is empty.")
        packets = list(self._packets)
        self._packets.clear()
        self._update_status_events()
        self.has_changed._trigger()
        return packets

In [6]:
edge = Edge(None, None, maxsize=3)

# Load data1
changed_event1 = edge.has_changed.subscribe()
edge._load(Packet("data1"))
assert changed_event1.is_set()

# Load data2
edge._load(Packet("data2"))

# Load data3
edge._load(Packet("data3"))
assert edge.is_full.get()

# Unload data1
assert await edge._unload().consume() == 'data1'
assert edge.num_packets() == 2

# Unload data3
assert await edge._unload_right().consume() == 'data3'
assert edge.num_packets() == 1

# Unload data2
assert await edge._unload().consume() == 'data2'
assert edge.num_packets() == 0

In [7]:
#|hide
show_doc(fbdev.net.Node)

---

### Node

>      Node (node_spec:fbdev.graph.NodeSpec, parent_net:fbdev.net.Net)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|exporti
async def _handle_edge_port_connection(is_input:bool, edge:Edge, port:BasePort):
    if is_input:
        edge_non_empty = edge.states.empty.get_state_event(False)
        while True:
            await await_multiple_events(port._requesting_packet, edge_non_empty)
            port._load_packet(edge._unload())
            port._received_packet.set()
    else:
        edge_non_full = edge.states.full.get_state_event(False)
        while True:
            await await_multiple_events(port._ready_to_unload, edge_non_full)
            edge._load(port._unload_packet())
            port._package_sent.set()

In [ ]:
#|exporti
async def _handle_component_config(component_process, component_executing_state, configured_state, config_port):
    """Continually awaits receiving packets at a config port, and updates the component process's config."""
    while True:
        config_payload = await config_port.receive()
        await component_executing_state.wait(False) # Doing the awaits in this order guarantees that a config wont be updated whilst the component is executing (though it can be received during it).
        component_process.set_config(config_port.name, config_payload)
        configured_state.set(component_process.check_configured())

In [8]:
#|export
class Node:
    def __init__(self, node_spec:NodeSpec, parent_net:'Net'):
        self._parent_net = parent_net
        self._node_spec = node_spec
        self._component_process = None
        
        self._states = StateCollection()
        self._states._add_state(StateHandler("initialised", False), readonly=True)
        self._states._add_state(StateHandler("configured", False), readonly=True)
        self._states._add_state(StateHandler("running", False), readonly=True)
        self._states._add_state(StateHandler("component_executing", False), readonly=True)

        self._events = EventCollection()
        self._events._add_event(EventHandler("stopped_running"))
        self._stop_running_event = asyncio.Event()

        self._edge_buses = {}
        self._config_port_handlers = {}
        
    @property
    def parent_net(self): return self._parent_net
    @property
    def node_spec(self): return self._node_spec
    @property
    def id(self): return self.node_spec.id
    @property
    def component_type(self): return self.node_spec.component_type
    @property
    def component_process(self): return self._component_process
    @property
    def states(self): return self._states
    @property
    def events(self): return self._events
    
    def _get_edge_ids(self):
        return [edge.id for port_type in self.node_spec for edge in self.node_spec.port_connections[port_type]]
    def _get_in_edge_ids(self):
        return [edge.id for port_type in self.node_spec for edge in self.node_spec.port_connections[port_type] if edge.is_input]
    def _get_out_edge_ids(self):
        return [edge.id for port_type in self.node_spec for edge in self.node_spec.port_connections[port_type] if not edge.is_input]
    
    def _initialise(self):
        self.component_process = self.component_type(self)
        self.states._initialised.set(True)
        
    def _start_edge_bus(self, edge_id):
        edge = self.parent_net.edges[edge_id]
        edge_spec = self.node_spec._parent_graph.edges[edge_id]
        if self == edge.tail_node:
            is_input, port_type, port_name = True, edge_spec.tail_port_type, edge_spec.tail_port_name
        elif self == edge.head_node:
            is_input, port_type, port_name = False, edge_spec.head_port_type, edge_spec.head_port_name
        else: raise RuntimeError("Edge is not connected to this node.")
        port = self.component_process.ports[port_type][port_name]
        if is_input: 
            edge_bus = asyncio.create_task(_handle_edge_port_connection(is_input, edge, port))
        else: 
            edge_bus = asyncio.create_task(_handle_edge_port_connection(is_input, edge, port))
        self._edge_buses[edge_id] = edge_bus
        
    def _stop_edge_bus(self, edge_id):
        self._edge_buses[edge_id].cancel()
        
    async def run(self):
        if not self.states.initialised.get(): raise RuntimeError("Node is not initialised.")
        self.running.set(True)
        self._stop_running_event.clear()
        
        # Start edge buses and config handlers
        for node_id in self._get_edge_ids(): self._start_edge_bus(node_id)
        config_tasks = []
        for edge_spec in self.node_spec.port_connections.config.values():
            port = self.component_process.config_inputs[edge_spec.head_port_name]
            config_task = asyncio.create_task(_handle_component_config(self.component_process, self.state.component_executing, self.state.configured, port))
            config_tasks.append(config_task)
        
        while not self._stop_running_event.is_set():
            await self.configured.wait(True)
            self.component_executing.set(True)
            await self.component_process.execute()
            self.component_executing.set(False)
        
        # Stop edge buses and config handlers
        for node_id in self._get_edge_ids(): self._stop_edge_bus(node_id)
        for config_task in config_tasks: config_task.cancel()
        
        self.running.set(False)
             
    async def stop(self):
        if not self.running.get(): raise RuntimeError("Node is not running.")
        self._stop_running_event.set()
        await self.running.wait(False)
        
    def __del__(self):
        if self.running.get():
            asyncio.run(self.stop())

In [9]:
#|hide
show_doc(fbdev.net.Net)

---

### Net

>      Net ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [10]:
#|export
class Net:
    def __init__(self, graph:Graph):
        self._graph = graph
        self._read_only_graph = ReadonlyGraph(graph)
        self._ports = None
        self._edges = {}
        self._nodes = {}
        
        self._setup_net()
        
    def _get_port_from_spec(self, port_spec: PortSpec, parent):
        port_args = [port_spec.name, port_spec.port_type, parent, port_spec.dtype, port_spec.data_validator]
        return InputPort(*port_args) if port_spec.is_input_port else OutputPort(*port_args)

    def _get_edge_from_spec(self, edge_spec: EdgeSpec):
        edge = Edge(maxsize=edge_spec.maxsize)

    def _add_net_port(self):
        pass

    def _setup_net(self):
        # Add net ports
        self._ports = PortCollection(self.graph.graph_port_specs, self)
        # Add edges
        for edge_spec in self.graph.edges.values():
            self._edges[edge_spec.id] = Edge(maxsize=edge_spec.maxsize)
        # Add nodes
        for node_spec in self.graph.nodes.values():
            self._nodes[node_spec.id] = Node(node_spec, self)
            
        
    
    @property
    def graph(self): return self._read_only_graph


In [ ]:
     
     
    def add_port(self, port_type: PortType, port_name: str, dtype=Optional[Type], data_validator=None, default=None, default_is_None=False):
        if port_name in self._graph_port_specs[port_type.label]:
            raise RuntimeError(f"Port {port_type.label}.{port_name} already exists.")
        self._graph_port_specs[port_type.label][port_name] = PortSpec(port_name, dtype, data_validator, default, default_is_None)
        self._graph_port_connections[port_type.label][port_name] = None
        
    def remove_port(self, port_type: PortType, port_name: str):
        if port_name not in self._graph_port_specs[port_type.label]:
            raise RuntimeError(f"Port {port_type.label}.{port_name} does not exist.")
        edge = self._graph_port_connections[port_type.label][port_name]
        if edge is not None:
            self.disconnect_edge_from_graph_port(port_type, port_name)
        del self._graph_port_specs[port_type.label][port_name]
        del self._graph_port_connections[port_type.label][port_name]
        
    def _check_node(self, node: NodeSpec):
        if node._parent_graph is None: self.add_node(node)
        if node._parent_graph != self: raise RuntimeError("Node is already in a graph.")
    def _check_edge(self, edge: EdgeSpec):
        if edge._parent_graph is None: self.add_edge(edge)
        if edge._parent_graph != self: raise RuntimeError("Edge is already in a graph.")
        
    def connect_edge_to_node(self, node, port_type: PortType, port_name: str, edge):
        if type(node) != NodeSpec: node = self.nodes[node]
        if type(edge) != EdgeSpec: edge = self.port_connections[edge]
        self._check_node(node)
        self._check_edge(edge)
        if node.port_connections[port_type.label][port_name] is not None: raise RuntimeError(f"Edge already connected to port {port_type.label}.{port_name}.")
        node.port_connections[port_type.label][port_name] = edge
        if port_type.is_input: edge._set_head(node, port_type, port_name)
        else: edge._set_tail(node, port_type, port_name)
        
    def disconnect_edge_from_node(self, node: Union[str, NodeSpec], port_type: PortType, port_name: str):
        if type(node) != NodeSpec: node = self.nodes[node]
        self._check_node(node)
        edge = node.port_connections[port_type.label][port_name]
        node.port_connections[port_type.label][port_name] = None
        if port_type.is_input: edge._clear_head()
        else: edge._clear_tail()
        
    def connect_edge_to_graph_port(self, port_type: PortType, port_name: str, edge:EdgeSpec):
        if type(edge) != EdgeSpec: edge = self.port_connections[edge]
        self._check_edge(edge)
        if self._graph_port_connections[port_type.label][port_name] is not None: raise RuntimeError(f"Edge already connected to port {port_type.label}.{port_name}.")
        self._graph_port_connections[port_type.label][port_name] = edge
        if port_type.is_input: edge._set_head(None, port_type, port_name)
        else: edge._set_tail(None, port_type, port_name)
        
    def disconnect_edge_from_graph_port(self, port_type: PortType, port_name: str):
        edge = self._graph_port_connections[port_type.label][port_name]
        self._graph_port_connections[port_type.label][port_name] = None
        if port_type.is_input: edge._clear_head()
        else: edge._clear_tail()
        
    def add_node(self, node, id=None):
        if node._parent_graph is not None: raise RuntimeError("Node is already in a graph.")
        node._id = id or len(self.nodes)
        if node._id in self.nodes: raise RuntimeError(f"Node id {node._id} already exists.")
        node._parent_graph = self
        self._nodes[node.id] = node
        
    def add_edge(self, edge: EdgeSpec, id=None):
        if edge._parent_graph is not None: raise RuntimeError("Edge is already in a graph.")
        edge._id = id or len(self.port_connections)
        edge._parent_graph = self
        self._edges[edge.id] = edge
        
    def remove_node(self, node: Union[str, NodeSpec]):
        if type(node) != NodeSpec: node = self.nodes[node]
        if node.graph != self: raise RuntimeError("Node is not in this graph.")
        node._graph = None
        for port_type in node.port_connections:
            for port_name in node.port_connections[port_type]:
                if node.port_connections[port_type][port_name] is not None:
                    self.disconnect_edge(node, PortType(port_type.upper()), port_name)
        node.port_connections._attrs.clear()
        
    def remove_edge(self, edge: EdgeSpec):
        if type(edge) != EdgeSpec: edge = self.port_connections[edge]
        if edge._parent_graph != self or edge.id not in self.port_connections: raise RuntimeError("Edge is not in this graph.")
        edge._id = None
        edge._parent_graph = None
        del self._edges[edge.id]
        
        if edge.tail_is_connected:
            if edge.tail_is_connected_to_graph:
                self.disconnect_edge_from_graph_port(edge.tail_port_type, edge.tail_port_name)
            else:
                self.disconnect_edge_from_node(edge.tail_target, edge.tail_port_type, edge.tail_port_name)
        if edge.head_is_connected:
            if edge.head_is_connected_to_graph:
                self.disconnect_edge_from_graph_port(edge.head_port_type, edge.head_port_name)
            else:
                self.disconnect_edge_from_node(edge.head_target, edge.head_port_type, edge.head_port_name)